In this notebook, we'll look at SIMD and threading support in Julia

In [2]:
A = rand(100_000)
A32 = rand(Float32, length(A)*2)

function simplesum(A)
    result = zero(eltype(A))
    for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

function simdsum(A)
    result = zero(eltype(A))
    @simd for i in eachindex(A)
        @inbounds result += A[i]
    end
    return result
end

simdsum (generic function with 1 method)

In [4]:
using BenchmarkTools

@btime sum($A)
@btime simplesum($A)
@btime simdsum($A)

@btime sum($A32)
@btime simplesum($A32)
@btime simdsum($A32)

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1277


  13.041 μs (0 allocations: 0 bytes)
  102.837 μs (0 allocations: 0 bytes)
  12.718 μs (0 allocations: 0 bytes)
  15.780 μs (0 allocations: 0 bytes)
  205.634 μs (0 allocations: 0 bytes)
  12.113 μs (0 allocations: 0 bytes)


99967.72f0

If `simdsum` is "faster", why not use it all the time?

In [5]:
simplesum(A), simdsum(A), sum(A)

(49911.125686407664, 49911.12568640862, 49911.12568640862)

In [6]:
simplesum(A32), simdsum(A32), sum(A32)

(99967.89f0, 99967.72f0, 99967.75f0)

How can we see if `@simd` is making good use of our CPU cores? One way would be a profiler, but we can also look directly from Julia:

In [7]:
@code_llvm simdsum(A32)


;  @ In[2]:12 within `simdsum'
define float @julia_simdsum_1618(%jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40)) {
top:
;  @ In[2]:14 within `simdsum'
; ┌ @ simdloop.jl:69 within `macro expansion'
; │┌ @ abstractarray.jl:212 within `eachindex'
; ││┌ @ abstractarray.jl:95 within `axes1'
; │││┌ @ abstractarray.jl:75 within `axes'
; ││││┌ @ array.jl:155 within `size'
       %1 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(11)*
       %2 = bitcast %jl_value_t addrspace(11)* %1 to %jl_value_t addrspace(10)* addrspace(11)*
       %3 = getelementptr inbounds %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)* addrspace(11)* %2, i64 3
       %4 = bitcast %jl_value_t addrspace(10)* addrspace(11)* %3 to i64 addrspace(11)*
       %5 = load i64, i64 addrspace(11)* %4, align 8
; ││││└
; ││││┌ @ tuple.jl:157 within `map'
; │││││┌ @ range.jl:323 within `OneTo' @ range.jl:314
; ││││││┌ @ promotion.jl:409 within `max'
         %6 = icmp sgt i64 %5, 0
       

In [8]:
@code_llvm simplesum(A32)


;  @ In[2]:4 within `simplesum'
define float @julia_simplesum_1623(%jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40)) {
top:
;  @ In[2]:6 within `simplesum'
; ┌ @ abstractarray.jl:212 within `eachindex'
; │┌ @ abstractarray.jl:95 within `axes1'
; ││┌ @ abstractarray.jl:75 within `axes'
; │││┌ @ array.jl:155 within `size'
      %1 = addrspacecast %jl_value_t addrspace(10)* %0 to %jl_value_t addrspace(11)*
      %2 = bitcast %jl_value_t addrspace(11)* %1 to %jl_value_t addrspace(10)* addrspace(11)*
      %3 = getelementptr inbounds %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)* addrspace(11)* %2, i64 3
      %4 = bitcast %jl_value_t addrspace(10)* addrspace(11)* %3 to i64 addrspace(11)*
      %5 = load i64, i64 addrspace(11)* %4, align 8
; │││└
; │││┌ @ tuple.jl:157 within `map'
; ││││┌ @ range.jl:323 within `OneTo' @ range.jl:314
; │││││┌ @ promotion.jl:409 within `max'
        %6 = icmp sgt i64 %5, 0
        %7 = select i1 %6, i64 %5, i64 0
; └└└└└└
  br i1 %6, l

We can also use threading in other ways. Julia can control the number of threads available to BLAS, or use threads ourselves!

In [9]:
using LinearAlgebra

A = rand(10_000, 2_000)
B = rand(2_000, 5_000)

BLAS.set_num_threads(1)
@btime $A*$B
BLAS.set_num_threads(4)
@btime $A*$B
# can set the number of threads even higher, of course. You can also use the environment variable 

  4.293 s (2 allocations: 381.47 MiB)
  1.783 s (2 allocations: 381.47 MiB)


10000×5000 Array{Float64,2}:
 503.971  494.632  486.423  512.715  …  495.348  494.287  490.079  495.776
 513.881  502.39   503.271  522.839     510.434  504.631  506.406  515.5
 516.066  502.328  504.748  515.33      508.983  505.34   504.395  513.875
 519.404  500.062  500.221  519.434     497.87   500.677  508.285  507.121
 508.663  495.219  490.846  516.207     495.062  491.511  497.319  507.443
 501.579  493.947  494.614  514.469  …  493.693  494.486  496.239  503.289
 497.671  491.556  486.265  509.046     493.358  490.073  492.431  503.061
 522.154  511.882  512.418  528.928     515.718  517.301  520.164  528.079
 503.844  496.853  493.93   514.278     493.035  495.016  493.673  499.037
 502.743  503.565  498.381  519.618     506.189  498.569  501.422  519.208
 506.564  495.665  498.885  516.443  …  505.6    503.638  506.618  514.4
 517.268  501.23   503.828  520.15      504.584  502.572  503.956  514.325
 509.655  504.053  496.453  517.45      501.475  498.986  502.946  510.81
 

But that's kind of boring... let's have some fun with threads ourselves.

In [13]:
using .Threads
nthreads()

1

In [ ]:
# a regular loop doesn't use threads by default
A = zeros(Int, nthreads())
for i in 1:nthreads()
    A[i] = threadid()
end
A

In [ ]:
# we need to use the @threads macro
A = zeros(Int, nthreads())
@threads for i in 1:nthreads()
    A[i] = threadid()
end
A

In [ ]:
function threaded_sum1(A)
    r = zero(eltype(A))
    @threads for i in eachindex(A)
        @inbounds r += A[i]
    end
    return r
end

A = rand(100_000)
@btime sum($A)
@btime threaded_sum1($A)
sum(A), threaded_sum1(A)

In [ ]:
function threaded_sum2(A)
    r = Atomic{eltype(A)}(zero(eltype(A)))
    @threads for i in eachindex(A)
        @inbounds atomic_add!(r, A[i])
    end
    return r[]
end
@btime sum($A)
@btime threaded_sum2($A)
sum(A), threaded_sum2(A)

In [ ]:
function threaded_sum3(A)
    R = zeros(eltype(A), nthreads())
    @threads for i in eachindex(A)
        @inbounds R[threadid()] += A[i]
    end
    r = zero(eltype(A))
    # sum the partial results from each thread
    for i in eachindex(R)
        @inbounds r += R[i]
    end
    return r
end

threaded_sum3(A)
@time threaded_sum3(A)